# Projeto 1 - Informed and adversarial search strategies
## Inteligência Artificial
### Grupo: 2
### Turma: PL5

- Diogo Filipe Gonçalves Cruz (202005947)
- Michel Castro Costa e Silva (202200437)
- Rodrigo Luz de Carvalho (20200184)

# Problema - Connect 4

**Connect Four** é um jogo de estratégia para dois jogadores. É jogado com 42 peças (21 de uma cor e 21 de outra) numa grade vertical com 7 colunas. Cada coluna pode conter no máximo 6 peças. Os dois jogadores jogam alternadamente colocando uma ficha numa das colunas. Quando uma ficha é colocada numa coluna, ela cai até atingir a parte inferior ou a ultima ficha nessa coluna. Um jogador vence quando pelo menos quatro de suas fichas estão alinhadas numa linha, coluna ou diagonal.

## Objetivo
O objetivo deste trabalho é implementar um algoritmo **A*** e **Monte Carlo Tree Search** (MCTS) para jogar este jogo. 

A celúla abaixo instala e importa a biblioteca numpy, necessaria pois usamos arrays do numpy para representar a board.

In [1]:
#
%pip install numpy
import numpy as np


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Definição de algumas constantes usadas durante o código
NUM_ROW = 6
NUM_COL = 7
EMPTY = "-"
PLAYER_PIECE = "X"
AI_PIECE = "O"

O código abaixo gera dois dicionarios usados no calcúlo das heuristicas de segmentos da board, ambos tem como chave uma posição da board, já os valores são:
- index_dict: Os indices dos segmentos na lista de heuristica, que terão suas heuristicas alteradas ao colocar uma peça nessa posição.
- pos_dict: Uma lista das listas de posições contidas no segmentos que contem a posição chave
abaixo tem um exemplo de ambos dicionarios

In [3]:
def get_segments(board):
        segments = []
        # Verifica linhas e colunas
        for i in range(NUM_ROW):
            line = board[i]
            for j in range(4):
                segments.append(line[j : j + 4])
            col = board[:, i]
            for j in range(3):
                segments.append(col[j : j + 4])

        # Verifica a ultima coluna
        col = board[:, NUM_COL - 1]
        for j in range(3):
            segments.append(col[j : j + 4])

        # Verifica as diagonais principais
        for i in range(-2, 4):
            dia = np.diag(board, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])

        # Dá flip no array e verifica as diagonais principais do array flipado
        # (equivalentes às diagonais perpendiculares às principais do array original)
        state_tr = np.fliplr(board)
        for i in range(-2, 4):
            dia = np.diag(state_tr, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])
        return segments

def gen_dict():
    board = [[f"{i},{j}" for j in range(7)]for i in range(6)]
    board = np.array(board)
    segments = get_segments(board)
    index_dict = {}
    position_dict = {}
    for j in range(7):
        for i in range(6):
            temp = []
            for z,segment in enumerate(segments):
                if f"{i},{j}" in segment:
                    temp.append(z)
                index_dict[(i,j)] = temp
                position_dict[(i,j)] = [[tuple(map(int, pos.split(","))) for pos in segments[index]] for index in temp]
    return index_dict,position_dict
index_dict, pos_dict = gen_dict()

In [4]:
print(index_dict[(1,2)])
print(pos_dict[(1,2)])

[7, 8, 9, 18, 19, 51, 52, 68]
[[(1, 0), (1, 1), (1, 2), (1, 3)], [(1, 1), (1, 2), (1, 3), (1, 4)], [(1, 2), (1, 3), (1, 4), (1, 5)], [(0, 2), (1, 2), (2, 2), (3, 2)], [(1, 2), (2, 2), (3, 2), (4, 2)], [(0, 1), (1, 2), (2, 3), (3, 4)], [(1, 2), (2, 3), (3, 4), (4, 5)], [(0, 3), (1, 2), (2, 1), (3, 0)]]


A seguir temos a definição da classe game, que representa um jogo, nela se tem os atributos:
- game_winner: armazena quem ganhou o jogo ou a peça vazia, caso ninguem tenha ganhado
- board_is_full: boolean que fica verdadeiro quando todas as colunas estão cheias
- segment_heuristics: lista da heuristica de cada segmento
- self.board: array que representa o tabuleiro
- self.last_move: guarda quem jogou por ultimo


In [5]:
class game:
    def __init__(self, calculate_heuristics=True):
        self.game_winner = EMPTY  # variáveis que controlam o fim do jogo
        self.board_is_full = False
        if calculate_heuristics: self.segment_heuristics = [0 for _ in range (69)] # Caso bool seja falso, não é necessario inicializar essa variavel
        self.board = np.full([NUM_ROW, NUM_COL], EMPTY)
        self.last_move = None
        self.calculate_heuristics = calculate_heuristics 
        self.first = PLAYER_PIECE #or AI_PIECE
        self.turn = 0

    def drawBoard(self):
        for i in range(7): print(i, end=" ") #imprime os numeros das colunas
        print() # coloca newline
        for line in np.flip(self.board, 0):
            for piece in line:
                print(piece, end=" ")
            print()# coloca newline
        return

    """Dado um input, verifica a função availableCollumns para saber se é válido.
    Se não for pede novamente o input, do contrário usa a função putGamePiece para alterar a board."""

    def playOneTurn(self):
        available = self.availableCollumns()
        try:  # error handling
            collumn = int(
                input("Choose in which collumn do you wanna play or press 9 to quit: ")
            )
        except:
            collumn = -1

        while collumn not in available:
            if collumn == 9:
                quit()  # opção para quitar
            try:  # more error handlings
                collumn = int(
                    input(
                        "The collumn you selected is either full or invalid, choose another one or press 9 to quit: "
                    )
                )
            except:
                collumn = -1
        self.putGamePiece(collumn, PLAYER_PIECE)
        return

    """Verifica qual a próxima row vazia e põe a peça nesta row. 
    Checa após cada movimento a função check_win_after_move para saber se houve ganhador.
    caso calculate_heuristics==True calcula as heuristicas dos segmentos alterados por aquele movimento
    """

    def putGamePiece(self, collumn, piece):
        available = self.availableCollumns()
        if collumn not in available:
            print("The last move was invalid, below is the last state of the board and the move made, the game will quit")
            self.drawBoard()
            print("last move = {collumn}")
            quit()

        piece_placement = self.nextEmptyRowinCollumn(collumn)
        self.board[piece_placement][collumn] = piece
        if self.check_win_after_move(piece_placement, collumn, piece):
            self.game_winner = piece  # checar se houve ganhador
        elif not self.availableCollumns():
            self.board_is_full = True  # se não há colunas vazias, a board está cheia
        # not list aparentemente é um dos jeitos mais eficientes de checar se uma lista está vazia, python é estranho - M
        if self.calculate_heuristics: self.update_heuristics(piece_placement, collumn) # atualiza as heuristicas se o bool for True
        self.last_move = collumn
        self.turn += 1
        return

    """Checa a última row para saber quais colunas não estão cheias. Retorna a lista de colunas."""

    def availableCollumns(self):
        available = []
        for i, value in enumerate(self.board[NUM_ROW - 1]):
            if value == EMPTY:
                available.append(i)
        return available

    """Dado uma coluna, verifica qual a próxima row vazia. Retorna o número da row."""

    def nextEmptyRowinCollumn(self, collumn):
        for i, value in enumerate(self.board[:, collumn]):
            if value == EMPTY:
                return i
        return

    """Função que checa se houve vitória após cada movimento. 
    Verifica somente os arrays que contém a peça em [move_row, move_col]."""
    
    def check_win_after_move(self, move_row, move_col, piece):
        win_segment = [piece, piece, piece, piece]
        for i in self.segments_that_intersect(move_row, move_col): # verifica se houve vitoria em todos os segmentos que são modificados pelo ultimo movimento
            if i == win_segment: return True
        return False

    # retorna lista com todos os segmentos de 4 em todas as direções
                
    def get_segments(self):
        segments = []
        # Verifica linhas e colunas
        for i in range(NUM_ROW):
            line = self.board[i]
            for j in range(4):
                segments.append(line[j : j + 4])
            col = self.board[:, i]
            for j in range(3):
                segments.append(col[j : j + 4])

        # Verifica a ultima coluna
        col = self.board[:, NUM_COL - 1]
        for j in range(3):
            segments.append(col[j : j + 4])

        # Verifica as diagonais principais
        for i in range(-2, 4):
            dia = np.diag(self.board, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])

        # Dá flip no array e verifica as diagonais principais do array flipado
        # (equivalentes às diagonais perpendiculares às principais do array original)
        state_tr = np.fliplr(self.board)
        for i in range(-2, 4):
            dia = np.diag(state_tr, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])
        return segments
    def segments_that_intersect(self, move_row, move_col):
        segments = [[self.board[pos[0],pos[1]] for pos in list] for list in pos_dict[(move_row,move_col)]]
        return segments

    # avalia um segmento e retorna a sua pontuação
    def update_heuristics(self,move_row,move_col):
        indexes = index_dict[(move_row,move_col)]
        segments = self.segments_that_intersect(move_row, move_col)             
        for i in range(len(indexes)):
            self.segment_heuristics[indexes[i]] = self.evaluate(segments[i])

    def evaluate(self, segment):
        count_x = 0
        count_o = 0

        for i in segment:
            if i == PLAYER_PIECE:
                count_x += 1
            elif i == AI_PIECE:
                count_o += 1

        if (count_x == 0 and count_o == 0) or (count_x > 0 and count_o > 0):
            return 0

        match count_x:
            case 1:
                return 1
            case 2:
                return 10
            case 3:
                return 50
            case 4:
                return 512

        match count_o:
            case 1:
                return -1
            case 2:
                return -10
            case 3:
                return -50
            case 4:
                return -512

    # avalia todos as posições para descobrir se há vencedor
    def evaluate_all(self):
        if self.game_winner == PLAYER_PIECE:
            return 512
        elif self.game_winner == AI_PIECE:
            return -512
        elif self.board_is_full:
            return 0
        else:
            sum = 0
            if self.player() == PLAYER_PIECE:
                sum = sum + 16
            elif self.player() == AI_PIECE:
                sum = sum - 16
            for segment in self.get_segments():
                sum = sum + self.evaluate(segment)
            return sum

    """Verifica se um determinado estado é um estado terminal/final 
    (estado em que um dos jogadores ganhou ou em que não há ações possíveis)"""

    def terminal(self):
        if self.game_winner == PLAYER_PIECE or self.game_winner == AI_PIECE:
            return True
        else:
            return self.board_is_full

    """Recebe um estado e retorna o vencedor (no caso deste existir)"""

    def checkwin_wholeboard(self):
        for (
            segment
        ) in self.get_segments():  # Itera sobre todos os segmentos de tamanho 4
            if np.array_equal(segment, ["X", "X", "X", "X"]):
                return "X"
            elif np.array_equal(segment, ["O", "O", "O", "O"]):
                return "O"
        return None

    #Função que retorna qual é o proximo jogador.

    def player(self):
        if self.board_is_full:
            return None
        
        elif self.first == AI_PIECE:
            if self.turn % 2 == 0:
                return AI_PIECE
            else:
                return PLAYER_PIECE
            
        elif self.first == PLAYER_PIECE:
            if self.turn % 2 == 0:
                return PLAYER_PIECE
            else:
                return AI_PIECE
            
        else:
            print("Problema na função player")
            quit()

    def utility(self):
        if self.game_winner == PLAYER_PIECE:
            return 512
        elif self.game_winner == AI_PIECE:
            return -512
        else:
            return 0
        
    def stop_calculate(self): # Para uso no mcts, faz com que parem de ser calculado as heuristicas
        self.calculate_heuristics = False
        delattr(self, "segment_heuristics")
        return

# A star

A heurística utilizada no A* foi a heurística fornecida no protocolo do trabalho. 
Se houver vencedor num estado o valor da heurística é:
- +512 se o vencedor for "X"
- -512 se o vencedor for "O"

Se não houver vencedor, o valor da heurística é o somatório do valor de todas as sequências de 4 peças, atribuindo-se os seguintes valores:
- 3 "X" -> +50
- 2 "X" -> +10
- 1 "X" -> +1
- Sem peças ou com peças dos dois jogadores -> 0
- 1 "O" -> -1
- 2 "O" -> -10
- 3 "O" -> -50

Somando-se um bonús:

- +16 se for a vez de "X" jogar
- -16 se for a vez de "O" jogar

In [6]:
import copy
class ai_aStar:
    def __init__(self) -> None:
        pass

    def get_move(self, state):

        actions = []
        heuristic = []
        for action in state.availableCollumns():
            new_state = copy.deepcopy(state)
            new_state.putGamePiece(action, "X")
            if new_state.game_winner != EMPTY:
                return heuristic, action
            
        for action in state.availableCollumns():
            new_state = copy.deepcopy(state)
            new_state.putGamePiece(action, "O")
            actions.append(action)
            heuristic.append(new_state.evaluate_all())
            
        index_min = np.argmin(heuristic)
        return heuristic, actions[index_min]

# MCTS

Na implementação do MCTS foi utilizado o algoritmo UCT (Upper Confidence Bound for Trees) para a seleção do nó a ser expandido. O valor de C foi ajustado manualmente de forma a otimizar os resultados.

In [7]:
import math
import random
import time #calcular tempo de execução

class MCTSNode:

    def __init__(self, state, parent=None) -> None:
        self.visits = 1
        self.value = 0
        self.state = state
        self.parent = parent
        self.children = []
        self.children_move = []

    def full_explored(self):
        return len(self.children) == len(self.state.availableCollumns())

    def update(self, v):
        self.value = self.value + v
        self.visits +=1
    
    def add_child(self, child, move):
        self.children.append(MCTSNode(child.state, parent=self))
        self.children_move.append(move)
    


class MonteCarloTreeSearch:
    def __init__(self, maxIter=10000, exploring_rate=10):
        self.maxIter = maxIter
        self.exploring_rate = exploring_rate
        
    def get_move(self, state):
        self.peca = state.player()
        new_state = copy.deepcopy(state)
        if new_state.calculate_heuristics: new_state.stop_calculate()
        node = MCTSNode(copy.deepcopy(new_state))
        for _ in range(self.maxIter):
            front = self.choose(node)
            value = self.rollout(front)
            self.backprop(front, value)
        #melhor =  self.bestChild(node)
        mv = float('-inf')
        for child in node.children:
            if child.value/child.visits > mv:
                mv = child.value/child.visits
                melhor = child
            print("----", child.value, child.visits, "...", child.state.last_move, child.value/child.visits)
        print("lolada")
        return 0,melhor.state.last_move # retorna 0 em [0] para manter o output consistente com as outras ais

    def choose(self, node):

        while not node.state.terminal():
            if not node.full_explored():
                return self.expand(node)
            else:
                node= self.bestChild(node)
        return node
    
    def expand(self, node):
        possible_actions = node.state.availableCollumns()
        not_used = []
        for action in possible_actions:
            if action not in node.children_move:
                not_used.append(action)
        
        ac = random.choice(not_used)
        new = copy.deepcopy(node.state)
        new.putGamePiece(ac,new.player())
        node.add_child(MCTSNode(new), ac)
        return node.children[-1]
    
    def bestChild(self, node):
        bestScore = float('-inf')
        bestChild = []
        for child in node.children:
            score = child.value/child.visits + self.exploring_rate * math.sqrt(2*math.log(node.visits)/child.visits)
            if score == bestScore:
                bestChild.append(child)
            if score> bestScore:
                bestChild = [child]
                bestScore = score
        return random.choice(bestChild)
    
    def rollout(self, node):
        new = node.state
        #print(node.state, new, new.state)
        while not new.terminal():
            new = copy.deepcopy(new)
            possible_actions = new.availableCollumns()
            if len(possible_actions) >0:
                new.putGamePiece(random.choice(possible_actions), new.player())
        
        if new.game_winner == self.peca:
            return 1
        else:
            return 0

    def backprop(self, node, value):
        while node != None:
            node.value = node.value + value
            node.visits = node.visits +1
            node = node.parent
        
        return 

# Modificação a explicar

In [8]:
class MCTSNode:

    def __init__(self, state, parent=None) -> None:
        self.visits = 1
        self.value = 0
        self.state = state
        self.parent = parent
        self.children = []
        self.children_move = []

    def full_explored(self):
        return len(self.children) == len(self.state.availableCollumns())

    def update(self, v):
        self.value = self.value + v
        self.visits +=1
    
    def add_child(self, child, move):
        self.children.append(MCTSNode(child.state, parent=self))
        self.children_move.append(move)
    


class MonteCarloTreeSearch_Mod:
    def __init__(self, maxIter=100000, exploring_rate=11):
        self.maxIter = maxIter
        self.exploring_rate = exploring_rate
        

    def get_move(self, state):
        self.peca = state.player()
        self.peca_do_outro = "X" if self.peca == "O" else "O"
        ad = state.availableCollumns()
        # Check for immediate wins for current player
        for i in ad:
            st = copy.deepcopy(state)
            st.putGamePiece(i, self.peca)
            if st.game_winner == self.peca:
                return 0, i
        
        # Check for immediate wins for opponent
        self.posicoes_vulneraveis = []
        for i in ad:
            st = copy.deepcopy(state)
            st.putGamePiece(i, self.peca_do_outro)
            if st.game_winner == self.peca_do_outro:
                self.posicoes_vulneraveis.append(i)
        time_start = time.time()
        node = MCTSNode(copy.deepcopy(state))
        for _ in range(self.maxIter):
            front = self.choose(node)
            value = self.rollout(front)
            self.backprop(front, value)
            if time.time() - time_start > 10:
                break
        #melhor =  self.bestChild(node)
    # Find the best move based on the highest average value per visit
        mv = float('-inf')
        melhor = node.children[0]
        segundo_melhor = node.children[0]
        for child in node.children:
            if child.visits > 0 and child.value / child.visits > mv:
                mv = child.value / child.visits
                segundo_melhor = melhor
                melhor = child
            elif child.visits > 0 and child.value / child.visits > segundo_melhor.value / segundo_melhor.visits:
                segundo_melhor = child
        
        # Print debug information
        for child in node.children:
            print("----", child.value, child.visits, "...", child.state.last_move, child.value/child.visits)
        #
        ## Check for low win rate but significant visits in children of the best move
        #for child in melhor.children:
        #    print("FILHO", child.state.last_move, " ----> ", child.value, child.visits, child.value/child.visits)
        #    if child.visits >= 0.1 * melhor.visits and child.value / child.visits == 0:
        #        print("Low win rate but significant visits in children of the best move.")
        #        print("Returning move corresponding to the second-best child.")
        #        return 0, segundo_melhor.state.last_move
        print(self.posicoes_vulneraveis)        
        if (melhor.state.last_move in self.posicoes_vulneraveis) and (segundo_melhor.state.last_move not in self.posicoes_vulneraveis):
            return 0, segundo_melhor.state.last_move
        return 0, melhor.state.last_move
    def choose(self, node):

        while not node.state.terminal():
            if not node.full_explored():
                return self.expand(node)
            else:
                node= self.bestChild(node)
        return node
    
    def expand(self, node):
        possible_actions = node.state.availableCollumns()
        not_used = []
        for action in possible_actions:
            if action not in node.children_move:
                not_used.append(action)
        
        ac = random.choice(not_used)
        new = copy.deepcopy(node.state)
        new.putGamePiece(ac,new.player())
        node.add_child(MCTSNode(new), ac)
        return node.children[-1]
    
    def bestChild(self, node):
        bestScore = float('-inf')
        bestChild = []
        for child in node.children:
            score = child.value/child.visits + self.exploring_rate * math.sqrt(2*math.log(node.visits)/child.visits)
            if score == bestScore:
                bestChild.append(child)
            if score> bestScore:
                bestChild = [child]
                bestScore = score
        return random.choice(bestChild)
    
    def rollout(self, node):
        new = node.state
        #print(node.state, new, new.state)
        while not new.terminal():
            new = copy.deepcopy(new)
            possible_actions = new.availableCollumns()
            if len(possible_actions) >0:
                new.putGamePiece(random.choice(possible_actions), new.player())
        
        if new.game_winner == self.peca:
            return 1
        else:
            return 0

    def backprop(self, node, value):
        while node != None:
            node.value = node.value + value
            node.visits = node.visits +1
            node = node.parent
        
        return 


# Extras - MiniMax e Alpha-Beta Pruning

Foi também implementado o algoritmo de MiniMax e Alpha-Beta Pruning para melhorar comparação de resultados. Neste algoritmo usamos a mesma heurística do A*.


In [9]:
class ai_miniMax:
    def __init__(self):
        pass


    def get_move(self, i_state, depth=3):
        state = copy.deepcopy(i_state)
        if depth == 0:
            return state.evaluate_all(), None
        
        if state.terminal():
            return state.utility(), None

        if state.player() == PLAYER_PIECE:
            v = -np.infty
            move = None
            for action in state.availableCollumns():
                new_state = copy.deepcopy(state)  
                new_state.putGamePiece(action, PLAYER_PIECE) 
                test = self.get_move(new_state, depth - 1)[0]
                if test > v:
                    v = test 
                    move = action
            return v, move
        
        else:
            v = np.infty
            move = None
            for action in state.availableCollumns():
                new_state = copy.deepcopy(state)  
                new_state.putGamePiece(action, AI_PIECE) 
                test = self.get_move(new_state, depth - 1)[0]
                if test < v:
                    v = test 
                    move = action
            return v, move

In [10]:
class ai_alphaBeta:
    def __init__(self):
      pass
      
    def get_move(self, state, depth=5, alpha=-np.infty, beta=np.infty):
        pl = state.player()
        if depth == 0:
            segments = state.get_segments()
            s = 0
            for segment in segments:
                s += state.evaluate(segment)
            
            if pl == "X":
                s += 16
            elif pl == "O":
                s -= 16
            return s, None

        if state.terminal():
            return state.utility(), None

        if pl == "X":
            v = -np.infty
            move = None
            for action in state.availableCollumns():
                new_state = copy.deepcopy(state)  
                new_state.putGamePiece(action, "X")
                test = self.get_move(new_state, depth - 1, alpha, beta)[0]
                if test > v:
                    v = test
                    move = action
                if v > beta:
                    break
                alpha = max(alpha, v)
            return v, move
        else:
            v = np.infty
            move = None
            for action in state.availableCollumns():
                new_state =  copy.deepcopy(state)  
                new_state.putGamePiece(action, "O")
                test = self.get_move(new_state, depth - 1, alpha, beta)[0]
                if test < v:
                    v = test
                    move = action
                if v < alpha:
                    break
                beta = min(beta, v)
            return v, move

# Função Main: Player vs AI

Ao executar este código pode-se escolher qual das AIs jogará contra o Player, e qual dos dois jogará a primeira jogada. A AI jogara com AI_PIECE ('O') e o player com PLAYER_PIECE ('X') independente de quem começou.

In [11]:
import os #poder usar função clear
from sys import platform #identificar plataforma
from IPython.display import clear_output

clear = lambda: clear_output(wait=True)

NEW_GAME = 1 #inicializa um novo jogo, e permite resetar (1) ou quitar (0)
CLEAR_TERMINAL = True #define se o terminal sera limpo ou não

if platform == "win32":
    clear = lambda: os.system('cls') #limpar o terminal do Windows; os.system('clear') para o Linux
else:
    clear = lambda: os.system('clear')

def clearTerminal():
    if CLEAR_TERMINAL:
        clear()

def show_heuristics(dgame):
    old = [dgame.evaluate(segment) for segment in dgame.get_segments()]
    print(old)
    print(dgame.segment_heuristics)

def playerMove(): #movimento do jogador
    novo_game.drawBoard()
    print("Next to play: Player\n")
    novo_game.playOneTurn()
    #show_heuristics(novo_game)
    clearTerminal()
    
def aiMove(): #movimento da ai
    novo_game.drawBoard()
    print(f"Next to play: {aiName}\n")
    ai_move = ai.get_move(novo_game)[1]
    if ai_move != None:
        novo_game.putGamePiece(ai_move,AI_PIECE)
    #show_heuristics(novo_game)
    clearTerminal()

def start_ai(): #inicializa a AI 1
    start = -1 # error handling
    while start not in range(5):
        try:  
            start = int(input("\nChoose the AI => 0 = A*, 1 = Mini Max, 2 = Alpha Beta, 3 = MCTS:, 4 = MCTS_Mod: "))
        except:
            continue
    match start:
        case 0:
            ai = ai_aStar()
            aiName = "A*"
        case 1:
            ai = ai_miniMax()
            aiName = "Mini Max"
        case 2:
            ai = ai_alphaBeta()
            aiName = "Alpha Beta"
        case 3:
            ai = MonteCarloTreeSearch()
            aiName = "MCTS"
        case 4:
            ai = MonteCarloTreeSearch_Mod()
            aiName = "MCTS_Mod"
    return ai,aiName

def first_player(): #decide quem vai primeiro
    try:  # error handling
        first = int(input("\nChoose 0 to go first or 1 to go second: "))
    except:
        first = -1
    if first not in range(2):
        first_player()
    return first

while NEW_GAME == 1: #Iniciando o game loop
     #inicia um novo objeto game
    clearTerminal()
    print("\nNew Game\n")
    
    start = start_ai()
    ai = start[0]
    aiName = start[1]
    if aiName == "MCTS":
        novo_game = game(calculate_heuristics=False)
    else:
        novo_game = game()
    
    if first_player() == 0:
        novo_game.first = PLAYER_PIECE
    else:
        novo_game.first = AI_PIECE
        
    clearTerminal()
    while novo_game.game_winner == EMPTY:
        if novo_game.player() == PLAYER_PIECE:
            playerMove()
        else:
            ti = time.time()
            aiMove()
            print(time.time()-ti)

    clearTerminal()
    novo_game.drawBoard()

    if novo_game.game_winner == AI_PIECE:
        print(f"\n{aiName} Won!")
    elif novo_game.game_winner == PLAYER_PIECE:
            print("\nPlayer Won!")
    else:
        print("\nIt's a tie!")
    NEW_GAME = int(input("\nType 0 to quit or 1 to play again: ")) #escolher se vai haver novo jogo

quit()


New Game



0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
Next to play: Player

0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
X - - - - - - 
Next to play: A*

0.065826416015625
0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
X - - O - - - 
Next to play: Player

0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
X - - - - - - 
X - - O - - - 
Next to play: A*

0.045426130294799805
0 1 2 3 4 5 6 
- - - - - - - 
- - - - - - - 
- - - - - - - 
- - - - - - - 
X - - - - - - 
X - - O O - - 
Next to play: Player



# Função Main: AI vs AI

Ao executar este código pode-se escolher quais AIs jogarão uma contra a outra, sendo que a primeira a ser escolhida fará a primeira jogada. A AI que iniciar jogará com PLAYER_PIECE ('X') e a outra com AI_PIECE ('O').

É possível definir através da varável NUM_GAMES quantos jogos serão jogados. No final é imprimido quantas vezes cada um ganhou, e em quantas jogadas.

In [11]:
from IPython.display import clear_output

CLEAR_TERMINAL = False #define se o terminal sera limpo ou não
NUM_GAMES = 5 #define o numero de jogos a serem jogados

clear = lambda: clear_output(wait=True)

def clearTerminal():
    if CLEAR_TERMINAL:
        clear()
    
def aiMove_1(): #movimento da ai 1
    ai_move = ai_1.get_move(novo_game)[1]
    if ai_move != None:
        novo_game.putGamePiece(ai_move,PLAYER_PIECE)

def aiMove_2(): #movimento da ai 2
    ai_move = ai_2.get_move(novo_game)[1]
    if ai_move != None:
        novo_game.putGamePiece(ai_move,AI_PIECE)
        
def start_ai(): #inicializa a AI 1
    start = -1 # error handling
    while start not in range(5):
        try:  
            start = int(input("\nChoose the AI => 0 = A*, 1 = Mini Max, 2 = Alpha Beta, 3 = MCTS, 4 = MCTS_Mod: "))
        except:
            continue
    match start:
        case 0:
            ai = ai_aStar()
            aiName = "A*"
        case 1:
            ai = ai_miniMax()
            aiName = "Mini Max"
        case 2:
            ai = ai_alphaBeta()
            aiName = "Alpha Beta"
        case 3:
            ai = MonteCarloTreeSearch()
            aiName = "MCTS"
        case 4:
            ai = MonteCarloTreeSearch_Mod()
            aiName = "MCTS_Mod"
    return ai,aiName

#-------- Game Loop --------

start = start_ai() #primeira ai
ai_1 = start[0]
ai_1_Name = start[1]
ai_1_wins = 0

start = start_ai() #segunda ai
ai_2 = start[0]
ai_2_Name = start[1]
ai_2_wins = 0

for i in range(1, NUM_GAMES + 1):
    clearTerminal()
    novo_game = game() #inicia um novo objeto game
    print(f"\nGame {i}\n")
        
    count_moves = 0
    while novo_game.game_winner == EMPTY and not novo_game.board_is_full:
        if novo_game.player() == PLAYER_PIECE:
            count_moves += 1
            aiMove_1()
            #novo_game.drawBoard()
        else:
            count_moves += 1
            aiMove_2()
            #novo_game.drawBoard()

    novo_game.drawBoard()

    if novo_game.game_winner == PLAYER_PIECE:
        ai_1_wins += 1
        print(f"\n{ai_1_Name} Won in {count_moves} moves!")
    elif novo_game.game_winner == AI_PIECE:
        ai_2_wins += 1
        print(f"\n{ai_2_Name} Won in {count_moves} moves!")
    else:
        print(f"\nIt's a tie in {count_moves} moves!")
        
print(f"\nResult:\n{ai_1_Name}: {ai_1_wins} vs {ai_2_Name}: {ai_2_wins}")
    
quit()



Game 4

||||||||||||||||||||
3
||||||||||||||||||||
||||||||||||||||||||
3
||||||||||||||||||||
||||||||||||||||||||
4
||||||||||||||||||||
||||||||||||||||||||
4
||||||||||||||||||||
||||||||||||||||||||
2
||||||||||||||||||||
||||||||||||||||||||
2
||||||||||||||||||||
||||||||||||||||||||
3
||||||||||||||||||||
||||||||||||||||||||
1
||||||||||||||||||||
||||||||||||||||||||
1
||||||||||||||||||||
||||||||||||||||||||
3
||||||||||||||||||||
||||||||||||||||||||


KeyboardInterrupt: 

# Testes

De forma a testar os algoritmos implementados, colocamos os diferentes algoritmos a jogar uns contra os outros. Os resultados obtidos foram os seguintes:

## A* a começar
 - A* ->
 - MCTS ->
 - MiniMax ->
 - Alpha-Beta Pruning ->
 - MCTS modificado ->

## MCTS a começar
- MCTS ->
- A* ->
- MiniMax ->
- Alpha-Beta Pruning ->
- MCTS modificado ->



De